<a href="https://colab.research.google.com/github/harryjakes/FYProject/blob/main/ht545_FinalYearProject_ColumnImputationGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


#MISGAN MR-NET Column imputation

In [ ]:
#All credit goes to original authors of misGAN : Steven Cheng-Xian Li, Benjamin M. Marlin

#Paper : Li, S.C., Jiang, B., & Marlin, B.M. (2019). MisGAN: Learning from Incomplete Data with Generative Adversarial Networks. ArXiv, abs/1902.09599.

#Project page (with all github links): https://github.com/steveli/misgan

In [ ]:
#To RUN

#Place the dataset of choice within the dataset folder, for example if using MRNet Coronal, this will look as follows:

#dataset>
#        MRNetCoronal>
#                     001.jpg...
#                     ... 
#                     xxx.jpg

#Once done, specify the column length to impute within the masked_image file. This line has been highlighted within the masked_image.py script itself.

#Then to run, simply run !python image_misgan_impute.py, results will apear within the results>MRNet> folder.

In [ ]:
!python image_misgan_impute.py

#MISGAN PVAE MR-NET Column imputation


In [ ]:
#All credit goes to original authors of P-VaE misGAN : Steven Cheng-Xian Li, Benjamin M. Marlin

#Paper : Li, S.C., & Marlin, B.M. (2020). Learning from Irregularly-Sampled Time Series: A Missing Data Perspective. ICML.
#Project page (with all github links): https://github.com/steveli/partial-encoder-decoder

In [ ]:
#To RUN

#Place the dataset of choice within the dataset folder, for example if using MRNet Coronal, this will look as follows:

#dataset>
#        MRNetCoronal>
#                     001.jpg...
#                     ... 
#                     xxx.jpg

#Once done, specify the column length to impute within the masked_image file. This line has been highlighted within the masked_image.py script itself.

#Then to run, simply run !python pvae.py, results will apear within the results>pvae> folder.

In [ ]:
!python celeba_pvae.py

#CYCLE GAN MR-NET Column imutation


In [ ]:
#All credit goes to original authors of cycleGAN : Jun-Yan Zhu, Taesung Park, Phillip Isola, Alexei A. Efros
#https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
#Paper : https://arxiv.org/pdf/1703.10593.pdf
#Project page (with all github links): https://junyanz.github.io/CycleGAN/

#For all pre-processing of databases, please refer to the cell found at the bottom of this notebook.
#For pre-trained paths, locate the paths according to the dataset that will be used, then place them into the output folder. 
#To run train or test with a certain database, !train.py --dataroot [dataset path goes here!] will suffice
#When testing, the output will be returned to the corrosponding A or B domain folder within output. 


#STEP BY STEP GUIDE
#Pre-process a database with some given mask, done in the cell found at the bottom of this notebook.
#Seperate this database into, training and testing sets. Each training and testing folder must contain a folder named A and B. 
#A will hold the masked images, B the ground truths. 
#The folders train and test can then be placed into the cycleGAN dataset folder.

#For example:
#dataset>
#         exampleSETA>
#                     testA>
#                          A (testing images from the dataset with masks)                          
#                     testB>
#                          B (testing images from the dataset of ground truths)
#                     trainA>
#                          A (training images from the dataset with masks)
#                     trainB>
#                          B (training images from the dataset of ground truths)
#                          
                         


#Once this is as follows, CycleGAN may be trained as follows (assuming we are in correct directory)
# !train.py --dataroot [dataset path goes here!] --model cycle_gan and likewise !test.py --dataroot [dataset path goes here!]  --model cycle_gan
#Please see train and test files for more variable changes (epochs, batch sizes, learning rate etc)
#

In [ ]:
!python train.py

#Metrics for evaluation

In [ ]:
#To run either the ssim or psnr metric, upon the generated outputs from the test set against their ground truths:

#Specify the dataset paths where each is held as follows:
generatedImages = glob.glob('some/path/to/generatedImages/*.jpg')
groundTruths = glob.glob('some/path/to/groundTruths/*.jpg')

#Then running sumOfScores(generatedImages,groundTruths) will return the sum of all ssims & psnrs for each pair.
#Dividing by the amount will give the average score for each as shown in the analysis section of the project paper.

In [ ]:
import cv2
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio
import scipy.misc as sm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import glob as glob

def readScaleConvert(first,second):
  a = cv2.imread(first)
  b = cv2.imread(second)
  a = cv2.resize(a, (250,250))
  b = cv2.resize(b, (250,250))
  a = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
  b = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
  return (a,b)

def ssim(a,b):
  score, null = structural_similarity(a,b, full=True)
  return score

def psnr(a,b):
  score = peak_signal_noise_ratio(a,b)
  return score

def sumOfScores(list_one,list_two):
  #return all ssim scores for two lists, don't have to be equal will just loop up to the shorter one
  ssims = [ssim(a,b) for a,b in [readScaleConvert(list_one[index],list_two[index]) for index in range(min(len(list_one),len(list_two)))]]
  psnrs = [psnr(a,b) for a,b in [readScaleConvert(list_one[index],list_two[index]) for index in range(min(len(list_one),len(list_two)))]]
  return sum(ssims),sum(psnrs)

In [ ]:
#For the FID evaluation

#Specify the paths to each dataset, generated images and ground truths. Then add these to the path variables within the fid_score script in the pytorch_fid folder.
#This can be found on line 254 & 256 and is highlighted in the script.

#The following lines must be run to import the correct files

!pip install pytorch-fid

import pytorch_fid

!python path/to/fid_score.py 


#Database Pre-processing (i.e 3-D --> 2-D & masking options)

In [ ]:
import glob
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image 
import random
import h5py

In [ ]:
#MRNet is available to download here : https://stanfordmlgroup.github.io/competitions/mrnet/
#These are the glob objects read in during the training, please replace this file name with wherever the downloaded database is stored.

coronal = glob.glob('path/to/MRNet-v1.0/MRNet-v1.0/train/coronal/*.npy')
sagittal = glob.glob('path/to/MRNet-v1.0/MRNet-v1.0/train/sagittal/*.npy')

In [ ]:
def createMaskAndGT(maskLength, databaseGlob, savePath, sliceNumber):
  #recommended maskLength is 48
  count = 0
  for scan in databaseGlob:
    image = np.rot90(np.load(scan)[sliceNumber])

    mask_start = random.randint(0,13)
    image[0:256,16*mask_start:maskLength+16*mask_start] = 0

    plt.imshow(image)
    plt.axis('off')
    newSavePath = savePath + "/A/%s.jpg"%(count)
    plt.savefig(newSavePath,bbox_inches='tight',pad_inches = 0) 

    image = np.rot90(np.load(scan)[sliceNumber])
    plt.imshow(image)
    plt.axis('off')
    newSavePath = savePath + "/B/%s.jpg"%(count)
    plt.savefig(newSavePath,bbox_inches='tight',pad_inches = 0)

    count+=1

In [ ]:
#Example, for the masking of 48 pixels:

createMaskAndGT(48,coronal,'path/to/saved/databases/coronal48',20)

#Then once this has been done, a split can be made witihin coronal48 into test & training folders.